Read Exported files using pandas

In [38]:
import pandas as pd
import statistics
lodFile = pd.read_excel("D:/Covid_data_New/Yr2023/Jan/Jan27/AN/20230126_SFSH_LOD_QC_AN Plate1_QS7_0011_20230126_165330.xlsx",sheet_name="Primary_result",skiprows=26)
lodFile = lodFile.replace(to_replace="UNDETERMINED",value=43)

d:\Python\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [39]:
targetThresholds={"VIC":38,"ABY":37,"FAM":38,"ALEXA 647":38,"JUN":33}
targets = dict()

for i in lodFile.index:
    cqValue = lodFile.loc[i,"Cq"]
    sampleName = lodFile.loc[i,"Sample"]
    reporterDye = lodFile.loc[i,'Reporter']
    if pd.isna(sampleName):
        continue
    if sampleName == "PC" or sampleName == "NC":
        continue
    string = sampleName+"\t"+reporterDye
    targets.setdefault(string,[]).append(cqValue)

writer = pd.ExcelWriter("D:/Covid_data_New/Yr2023/Jan/Jan30/a.xlsx")
df_list = pd.DataFrame()
text = open("D:/Covid_data_New/Yr2023/Jan/Jan27/AN/a.txt","a")

for d in targets:
    s = d.split("\t")
    clear = 0
    strong = 0
    total = 0
    tpr = 0
    data=[]
    
    for v in targets[d]:
        total+=1
        if v < 43:
            data.append(v)
        if v < targetThresholds[s[1]]:
            clear +=1
        tpr = float(clear/total) * 100
    cqMaen = 0
    cqSd = 0
    if data:
        cqMaen =  "%.3f" % statistics.mean(data)
        cqSd =  "%.3f" % statistics.stdev(data)
    if tpr != 0:
        tpr = "%.2f" % tpr
    text.write(s[0]+"\t"+s[1]+"\t"+str(clear)+"\t"+str(total)+"\t"+str(tpr)+"\t"+str(cqMaen)+"\t"+str(cqSd)+"\n")
    listL=[clear,total,tpr,cqMaen,cqSd]
    dfx = pd.DataFrame([listL],index=[s[0]],columns=['Clean','Total','TPR','Mean','SD'])
    df_list = pd.concat([df_list,dfx])

df_list.to_excel(writer,sheet_name = "Data1")
writer.close()
text.close()
